# PAROL6 机械臂完整功能测试教程

## 📋 教程说明

这个 Jupyter Notebook 将带你逐步测试 PAROL6 机械臂的所有功能。

### 使用方法：
1. **按顺序执行**：从上到下依次运行每个单元格（Cell）
2. **观察结果**：每个单元格都会显示执行结果和状态
3. **遇到错误**：查看错误信息，根据提示排查问题
4. **安全第一**：运动测试前确保机器人周围无障碍物

### 前提条件：
- ✅ 机器人已通过 USB 连接到电脑
- ✅ 机器人电源已开启
- ✅ 已启动 `headless_commander.py` 服务器
- ✅ 急停按钮可随时触及

### 测试阶段：
1. 环境检查
2. 基础连接测试
3. 基础运动测试
4. 夹爪控制测试
5. 高级运动测试
6. 视觉系统测试
7. 综合应用测试

---

## 🔧 阶段 1：环境检查

### 1.1 检查 Python 依赖

这个单元格会检查所有必需的 Python 库是否已安装。

In [ ]:
# 检查所有必需的依赖
import sys

def check_module(name, package_name=None):
    """检查模块是否安装"""
    if package_name is None:
        package_name = name
    try:
        __import__(name)
        print(f"✓ {package_name:30s} - 已安装")
        return True
    except ImportError:
        print(f"✗ {package_name:30s} - 未安装")
        return False

print("=" * 70)
print("检查核心依赖（服务器端必需）")
print("=" * 70)

core_modules = [
    ('roboticstoolbox', 'roboticstoolbox-python'),
    ('numpy', 'numpy'),
    ('scipy', 'scipy'),
    ('spatialmath', 'spatialmath'),
    ('serial', 'pyserial'),
]

core_ok = all(check_module(name, pkg) for name, pkg in core_modules)

print("\n" + "=" * 70)
print("检查视觉依赖（视觉功能必需）")
print("=" * 70)

vision_modules = [
    ('cv2', 'opencv-python'),
    ('pyrealsense2', 'pyrealsense2'),
    ('PIL', 'Pillow'),
]

vision_ok = all(check_module(name, pkg) for name, pkg in vision_modules)

print("\n" + "=" * 70)
print("总结")
print("=" * 70)

if core_ok:
    print("✓ 核心依赖完整 - 可以进行基础功能测试")
else:
    print("✗ 核心依赖缺失")
    print("  安装命令: pip3 install roboticstoolbox-python==1.0.3 numpy==1.23.4 scipy spatialmath pyserial")

if vision_ok:
    print("✓ 视觉依赖完整 - 可以进行视觉功能测试")
else:
    print("⚠ 视觉依赖缺失 - 视觉功能将不可用")
    print("  安装命令: pip3 install opencv-python pyrealsense2 Pillow")

### 1.2 导入机器人 API

导入所有需要的控制函数。

In [ ]:
# 添加 Headless 目录到路径
import sys
import os

# 获取当前工作目录
current_dir = os.getcwd()
headless_path = os.path.join(current_dir, 'Headless')

if headless_path not in sys.path:
    sys.path.insert(0, headless_path)

print(f"当前工作目录: {current_dir}")
print(f"Headless 路径: {headless_path}")

# 导入所有机器人控制函数
try:
    from robot_api import *
    import time
    print("\n✓ 成功导入机器人 API")
    print("\n可用的主要函数:")
    print("  - home_robot()              # 机器人回零")
    print("  - move_robot_joints()       # 关节运动")
    print("  - move_robot_pose()         # 位置运动")
    print("  - move_robot_cartesian()    # 直线运动")
    print("  - smooth_circle()           # 圆形轨迹")
    print("  - control_electric_gripper() # 夹爪控制")
    print("  - get_robot_pose()          # 查询位置")
    print("  - get_robot_joint_angles()  # 查询角度")
except ImportError as e:
    print(f"✗ 导入失败: {e}")
    print("\n请检查:")
    print("  1. 确保在项目根目录下运行此 notebook")
    print("  2. Headless/robot_api.py 文件是否存在")

---

## 🔌 阶段 2：基础连接测试

### 2.1 测试与机器人服务器的连接

⚠️ **重要**: 在运行此单元格之前，请确保：
1. 机器人已开机
2. 已在另一个终端运行 `python Headless/headless_commander.py`
3. 看到 "Listening for UDP commands on port 5001" 消息

In [ ]:
# 测试基础连接
print("=" * 70)
print("测试 1：检查机器人连接状态")
print("=" * 70)

# 测试 1：查询机器人是否停止
print("\n[1/5] 查询机器人运动状态...")
try:
    is_stopped = is_robot_stopped()
    if is_stopped is not None:
        print(f"✓ 连接成功！机器人状态: {'已停止' if is_stopped else '运动中'}")
    else:
        print("✗ 无法获取机器人状态")
        print("\n请检查:")
        print("  1. headless_commander.py 是否正在运行？")
        print("  2. 防火墙是否阻止了 UDP 5001 端口？")
except Exception as e:
    print(f"✗ 连接失败: {e}")
    print("\n无法继续后续测试，请先解决连接问题")

time.sleep(0.3)

### 2.2 查询机器人状态信息

In [ ]:
# 查询关节角度
print("[2/5] 查询当前关节角度...")
angles = get_robot_joint_angles()
if angles:
    print("✓ 当前关节角度:")
    for i, angle in enumerate(angles, 1):
        print(f"   关节 {i}: {angle:7.2f}°")
else:
    print("✗ 无法获取关节角度")

time.sleep(0.3)

In [ ]:
# 查询末端位置
print("[3/5] 查询末端执行器位置...")
pose = get_robot_pose()
if pose:
    print("✓ 当前位置姿态:")
    print(f"   位置 (mm):  X={pose[0]:7.2f}, Y={pose[1]:7.2f}, Z={pose[2]:7.2f}")
    print(f"   姿态 (deg): Rx={pose[3]:7.2f}, Ry={pose[4]:7.2f}, Rz={pose[5]:7.2f}")
else:
    print("✗ 无法获取位置姿态")

time.sleep(0.3)

In [ ]:
# 查询 IO 状态
print("[4/5] 查询 I/O 状态...")
io_status = get_robot_io()
if io_status:
    print("✓ I/O 状态:")
    print(f"   输入 1: {io_status[0]}  输入 2: {io_status[1]}")
    print(f"   输出 1: {io_status[2]}  输出 2: {io_status[3]}")
    print(f"   急停状态: {'⚠️ 按下' if io_status[4] else '✓ 释放'}")
    
    if io_status[4]:
        print("\n⚠️ 警告: 急停按钮被按下！")
        print("   请释放急停按钮，然后在服务器终端按 'e' 键重新启用机器人")
else:
    print("✗ 无法获取 I/O 状态")

time.sleep(0.3)

In [ ]:
# 查询完整状态
print("[5/5] 查询完整机器人状态...")
status = get_robot_status()
if status:
    print("✓ 完整状态获取成功")
    print(f"   运动状态: {'已停止' if status['stopped'] else '运动中'}")
    print(f"   急停状态: {'按下' if status['estop'] else '释放'}")
    print(f"   位置: X={status['pose'][0]:.1f}, Y={status['pose'][1]:.1f}, Z={status['pose'][2]:.1f}")
else:
    print("✗ 无法获取完整状态")

print("\n" + "=" * 70)
print("✓ 基础连接测试完成！")
print("=" * 70)

---

## 🤖 阶段 3：基础运动测试

⚠️ **安全提醒**：
- 确保机器人周围无障碍物
- 保持急停按钮可触及
- 首次运动时站在机器人侧面观察
- 发现异常立即按下急停

### 3.1 机器人回零测试

In [ ]:
# 机器人回零
print("=" * 70)
print("测试：机器人回零（Homing）")
print("=" * 70)
print("\n机器人将移动到零位（所有关节归零）")
print("⚠️  请确保周围无障碍物！")
print("\n按 Shift+Enter 继续执行...")

# 记录开始时间
start_time = time.time()

# 执行回零
print("\n→ 开始回零...")
result = home_robot(wait_for_ack=True, timeout=30)

# 计算耗时
elapsed = time.time() - start_time

if result and result.get('status') == 'COMPLETED':
    print(f"✓ 回零完成！耗时: {elapsed:.2f} 秒")
    
    # 验证位置
    time.sleep(0.5)
    angles = get_robot_joint_angles()
    print("\n当前关节角度:")
    for i, angle in enumerate(angles, 1):
        print(f"  关节 {i}: {angle:7.2f}°")
else:
    print("✗ 回零失败")
    if result:
        print(f"  状态: {result.get('status')}")

### 3.2 单关节运动测试

测试单个关节的运动控制。机器人底座将缓慢旋转。

In [ ]:
# 单关节运动测试
print("=" * 70)
print("测试：单关节运动（底座旋转）")
print("=" * 70)
print("\n机器人底座将执行以下动作:")
print("  0° → 45° → -45° → 0°")
print("\n运动速度: 30%（缓慢安全）")

# 记录初始位置
initial_angles = get_robot_joint_angles()
print(f"\n初始位置: 关节1 = {initial_angles[0]:.2f}°")

# 步骤 1: 移动到 45°
print("\n→ [步骤 1/3] 移动到 45°...")
move_robot_joints([45, -90, 180, 0, 0, 180], speed_percentage=30, wait_for_ack=True)
time.sleep(0.5)
current = get_robot_joint_angles()
print(f"✓ 到达: 关节1 = {current[0]:.2f}°")

time.sleep(1)

# 步骤 2: 移动到 -45°
print("\n→ [步骤 2/3] 移动到 -45°...")
move_robot_joints([-45, -90, 180, 0, 0, 180], speed_percentage=30, wait_for_ack=True)
time.sleep(0.5)
current = get_robot_joint_angles()
print(f"✓ 到达: 关节1 = {current[0]:.2f}°")

time.sleep(1)

# 步骤 3: 返回初始位置
print("\n→ [步骤 3/3] 返回初始位置...")
move_robot_joints(initial_angles, speed_percentage=30, wait_for_ack=True)
time.sleep(0.5)
current = get_robot_joint_angles()
print(f"✓ 返回: 关节1 = {current[0]:.2f}°")

print("\n✓ 单关节运动测试完成！")

### 3.3 多关节协调运动测试

测试多个关节的协调运动能力。

In [ ]:
# 多关节协调运动
print("=" * 70)
print("测试：多关节协调运动")
print("=" * 70)
print("\n机器人将执行一个涉及多个关节的姿态变换")
print("运动时长: 3.0 秒")

# 定义测试姿态
test_pose = [30, -60, 150, 0, -30, 180]
print(f"\n目标姿态: {test_pose}")

# 执行运动
print("\n→ 开始运动...")
move_robot_joints(test_pose, duration=3.0, wait_for_ack=True)
time.sleep(0.5)

# 验证到达位置
current = get_robot_joint_angles()
print("\n✓ 运动完成")
print(f"  目标: {[round(a, 1) for a in test_pose]}")
print(f"  实际: {[round(a, 1) for a in current]}")

# 计算误差
errors = [abs(t - c) for t, c in zip(test_pose, current)]
max_error = max(errors)
print(f"  最大误差: {max_error:.2f}°")

if max_error < 2.0:
    print("\n✓ 精度良好（误差 < 2°）")
else:
    print(f"\n⚠ 精度一般（误差 = {max_error:.2f}°）")

# 返回零位
time.sleep(1)
print("\n→ 返回零位...")
home_robot(wait_for_ack=True)
print("✓ 已返回零位")

### 3.4 笛卡尔空间运动测试

测试在笛卡尔坐标系中的位置控制。

In [ ]:
# 笛卡尔空间运动
print("=" * 70)
print("测试：笛卡尔空间运动")
print("=" * 70)
print("\n机器人末端将移动到几个不同的笛卡尔位置")

# 获取起始位置
start_pose = get_robot_pose()
print(f"\n起始位置: X={start_pose[0]:.1f}, Y={start_pose[1]:.1f}, Z={start_pose[2]:.1f}")

# 定义测试位置序列
test_positions = [
    [250, 0, 200, 180, 0, 0],
    [250, 50, 200, 180, 0, 0],
    [200, 50, 150, 180, 0, 0]
]

# 执行运动序列
for i, pos in enumerate(test_positions, 1):
    print(f"\n→ [位置 {i}/{len(test_positions)}] 移动到 [{pos[0]}, {pos[1]}, {pos[2]}]...")
    move_robot_pose(pos, speed_percentage=40, wait_for_ack=True)
    time.sleep(0.5)
    
    current = get_robot_pose()
    print(f"✓ 到达: X={current[0]:.1f}, Y={current[1]:.1f}, Z={current[2]:.1f}")
    
    # 计算位置误差
    error = ((pos[0]-current[0])**2 + (pos[1]-current[1])**2 + (pos[2]-current[2])**2)**0.5
    print(f"  位置误差: {error:.2f} mm")
    
    time.sleep(1)

print("\n✓ 笛卡尔空间运动测试完成！")

### 3.5 笛卡尔直线运动测试

测试保证直线路径的笛卡尔运动。

In [ ]:
# 笛卡尔直线运动
print("=" * 70)
print("测试：笛卡尔直线运动")
print("=" * 70)
print("\n机器人末端将沿直线路径运动")
print("这个功能保证工具中心点(TCP)沿直线移动")

# 移动到起点
start_point = [200, -50, 180, 180, 0, 0]
end_point = [200, 50, 180, 180, 0, 0]

print(f"\n起点: Y = {start_point[1]} mm")
print(f"终点: Y = {end_point[1]} mm")
print(f"直线距离: {abs(end_point[1] - start_point[1])} mm")

# 移动到起点
print("\n→ 移动到起点...")
move_robot_cartesian(start_point, duration=3.0, wait_for_ack=True)
time.sleep(0.5)
print("✓ 到达起点")

time.sleep(1)

# 执行直线运动到终点
print("\n→ 执行直线运动到终点...")
print("  （观察：末端应该沿直线移动，无弧线）")
move_robot_cartesian(end_point, duration=4.0, wait_for_ack=True)
time.sleep(0.5)
print("✓ 到达终点")

time.sleep(1)

# 直线返回起点
print("\n→ 直线返回起点...")
move_robot_cartesian(start_point, duration=4.0, wait_for_ack=True)
time.sleep(0.5)
print("✓ 返回起点")

print("\n✓ 笛卡尔直线运动测试完成！")

### 3.6 点动控制测试

测试笛卡尔坐标系下的点动（微调）功能。

In [ ]:
# 点动控制测试
print("=" * 70)
print("测试：笛卡尔点动控制")
print("=" * 70)
print("\n机器人将在笛卡尔空间进行小幅度点动")
print("点动速度: 30%")
print("点动时长: 1.0 秒/次（约 20mm 距离）")

# 记录起始位置
start_pos = get_robot_pose()
print(f"\n起始位置: X={start_pos[0]:.1f}, Y={start_pos[1]:.1f}, Z={start_pos[2]:.1f}")

# Z 轴正向点动（向上）
print("\n→ [1/4] Z 轴正向点动（向上）...")
jog_cartesian(frame='WRF', axis='Z+', speed_percentage=30, duration=1.0, wait_for_ack=True)
time.sleep(0.5)
current = get_robot_pose()
delta_z = current[2] - start_pos[2]
print(f"✓ 完成，Z 轴移动: {delta_z:.1f} mm")
time.sleep(1)

# X 轴正向点动（向前）
print("\n→ [2/4] X 轴正向点动（向前）...")
before_x = current[0]
jog_cartesian(frame='WRF', axis='X+', speed_percentage=30, duration=1.0, wait_for_ack=True)
time.sleep(0.5)
current = get_robot_pose()
delta_x = current[0] - before_x
print(f"✓ 完成，X 轴移动: {delta_x:.1f} mm")
time.sleep(1)

# X 轴负向点动（向后，返回）
print("\n→ [3/4] X 轴负向点动（向后）...")
jog_cartesian(frame='WRF', axis='X-', speed_percentage=30, duration=1.0, wait_for_ack=True)
time.sleep(0.5)
current = get_robot_pose()
print(f"✓ 完成，当前 X: {current[0]:.1f} mm")
time.sleep(1)

# Z 轴负向点动（向下，返回）
print("\n→ [4/4] Z 轴负向点动（向下）...")
jog_cartesian(frame='WRF', axis='Z-', speed_percentage=30, duration=1.0, wait_for_ack=True)
time.sleep(0.5)
current = get_robot_pose()
print(f"✓ 完成，当前 Z: {current[2]:.1f} mm")

# 验证返回
final_pos = get_robot_pose()
print(f"\n最终位置: X={final_pos[0]:.1f}, Y={final_pos[1]:.1f}, Z={final_pos[2]:.1f}")
print(f"位置偏差: X={abs(final_pos[0]-start_pos[0]):.1f}, Y={abs(final_pos[1]-start_pos[1]):.1f}, Z={abs(final_pos[2]-start_pos[2]):.1f} mm")

print("\n✓ 点动控制测试完成！")

---

## 🦾 阶段 4：夹爪控制测试

### 4.1 检测夹爪类型

首先确认你使用的夹爪类型。

In [ ]:
# 夹爪类型选择
print("=" * 70)
print("夹爪类型选择")
print("=" * 70)
print("\n请设置你的夹爪类型:")
print("  'electric'  - 电动夹爪（精确位置控制）")
print("  'pneumatic' - 气动夹爪（开/关控制）")

# 设置夹爪类型（修改下面这一行）
GRIPPER_TYPE = 'electric'  # 或 'pneumatic'

print(f"\n已选择: {GRIPPER_TYPE} 夹爪")

if GRIPPER_TYPE == 'electric':
    print("\n电动夹爪支持的功能:")
    print("  - 精确位置控制 (0-255)")
    print("  - 速度控制 (0-255)")
    print("  - 力控制 (100-1000 mA)")
    print("  - 物体检测")
    print("  - 状态反馈")
elif GRIPPER_TYPE == 'pneumatic':
    print("\n气动夹爪支持的功能:")
    print("  - 开/关控制")
    print("  - 数字输出端口控制")
    PNEUMATIC_PORT = 1  # 设置气动夹爪使用的端口（1 或 2）
    print(f"  - 使用端口: {PNEUMATIC_PORT}")

### 4.2 电动夹爪测试

如果你使用电动夹爪，运行此单元格。

In [ ]:
# 电动夹爪测试（仅当 GRIPPER_TYPE == 'electric' 时运行）
if GRIPPER_TYPE == 'electric':
    print("=" * 70)
    print("测试：电动夹爪控制")
    print("=" * 70)
    
    # 测试 1: 校准夹爪
    print("\n[1/5] 校准夹爪...")
    print("  夹爪将完全打开并校准零点")
    control_electric_gripper(action='calibrate', wait_for_ack=True)
    time.sleep(2)
    
    status = get_electric_gripper_status(verbose=False)
    if status:
        print(f"✓ 校准完成")
        print(f"  当前位置: {status[1]}")
    time.sleep(1)
    
    # 测试 2: 完全打开
    print("\n[2/5] 完全打开夹爪...")
    control_electric_gripper(action='move', position=0, speed=100, wait_for_ack=True)
    time.sleep(2)
    
    status = get_electric_gripper_status(verbose=False)
    if status:
        print(f"✓ 夹爪已打开")
        print(f"  当前位置: {status[1]}")
    time.sleep(1)
    
    # 测试 3: 完全关闭
    print("\n[3/5] 完全关闭夹爪...")
    control_electric_gripper(action='move', position=255, speed=100, current=500, wait_for_ack=True)
    time.sleep(2)
    
    status = get_electric_gripper_status(verbose=False)
    if status:
        print(f"✓ 夹爪已关闭")
        print(f"  当前位置: {status[1]}")
        print(f"  物体检测: {'检测到物体' if status[5] else '未检测到物体'}")
    time.sleep(1)
    
    # 测试 4: 中间位置
    print("\n[4/5] 移动到中间位置 (128)...")
    control_electric_gripper(action='move', position=128, speed=150, wait_for_ack=True)
    time.sleep(2)
    
    status = get_electric_gripper_status(verbose=False)
    if status:
        print(f"✓ 到达中间位置")
        print(f"  当前位置: {status[1]}")
    time.sleep(1)
    
    # 测试 5: 打开准备下次使用
    print("\n[5/5] 打开夹爪准备下次使用...")
    control_electric_gripper(action='move', position=0, speed=100, wait_for_ack=True)
    time.sleep(2)
    print("✓ 夹爪已打开")
    
    # 显示完整状态
    print("\n" + "=" * 70)
    print("完整夹爪状态:")
    print("=" * 70)
    get_electric_gripper_status(verbose=True)
    
    print("\n✓ 电动夹爪测试完成！")
    
else:
    print("跳过电动夹爪测试（当前选择: pneumatic）")

### 4.3 气动夹爪测试

如果你使用气动夹爪，运行此单元格。

In [ ]:
# 气动夹爪测试（仅当 GRIPPER_TYPE == 'pneumatic' 时运行）
if GRIPPER_TYPE == 'pneumatic':
    print("=" * 70)
    print("测试：气动夹爪控制")
    print("=" * 70)
    print(f"\n使用数字输出端口: {PNEUMATIC_PORT}")
    
    # 测试 1: 打开夹爪
    print("\n[1/3] 打开气动夹爪...")
    control_pneumatic_gripper(action='open', port=PNEUMATIC_PORT, wait_for_ack=True)
    time.sleep(1)
    
    io_status = get_robot_io(verbose=False)
    if io_status:
        output_state = io_status[PNEUMATIC_PORT + 1]
        print(f"✓ 打开命令已发送")
        print(f"  输出端口 {PNEUMATIC_PORT} 状态: {output_state}")
    time.sleep(2)
    
    # 测试 2: 关闭夹爪
    print("\n[2/3] 关闭气动夹爪...")
    control_pneumatic_gripper(action='close', port=PNEUMATIC_PORT, wait_for_ack=True)
    time.sleep(1)
    
    io_status = get_robot_io(verbose=False)
    if io_status:
        output_state = io_status[PNEUMATIC_PORT + 1]
        print(f"✓ 关闭命令已发送")
        print(f"  输出端口 {PNEUMATIC_PORT} 状态: {output_state}")
    time.sleep(2)
    
    # 测试 3: 再次打开
    print("\n[3/3] 再次打开夹爪...")
    control_pneumatic_gripper(action='open', port=PNEUMATIC_PORT, wait_for_ack=True)
    time.sleep(1)
    print("✓ 夹爪已打开")
    
    # 显示 IO 状态
    print("\n" + "=" * 70)
    print("当前 I/O 状态:")
    print("=" * 70)
    get_robot_io(verbose=True)
    
    print("\n✓ 气动夹爪测试完成！")
    
else:
    print("跳过气动夹爪测试（当前选择: electric）")

---

## 🎨 阶段 5：高级运动测试

### 5.1 圆形轨迹测试

测试机器人执行平滑圆形轨迹的能力。

In [ ]:
# 准备高级运动测试
print("=" * 70)
print("准备高级运动测试")
print("=" * 70)
print("\n移动机器人到测试起始位置...")

# 移动到适合测试的姿态
move_robot_joints([0, -90, 180, 0, -45, 180], duration=3.0, wait_for_ack=True)
time.sleep(1)
print("✓ 准备完成")

In [ ]:
# 圆形轨迹测试
print("=" * 70)
print("测试：圆形轨迹")
print("=" * 70)
print("\n机器人末端将画一个水平圆形")
print("  圆心: [220, 0, 180] mm")
print("  半径: 40 mm")
print("  平面: XY（水平面）")
print("  时长: 8.0 秒")
print("\n观察: 末端应该画出完美的圆形轨迹")

# 记录开始时间
start_time = time.time()

# 执行圆形轨迹
print("\n→ 开始执行圆形轨迹...")
try:
    smooth_circle(
        center=[220, 0, 180],
        radius=40,
        plane='XY',
        duration=8.0,
        clockwise=False,
        wait_for_ack=True
    )
    
    elapsed = time.time() - start_time
    print(f"\n✓ 圆形轨迹完成！实际耗时: {elapsed:.2f} 秒")
    
    # 验证返回位置
    time.sleep(0.5)
    final_pose = get_robot_pose()
    print(f"最终位置: X={final_pose[0]:.1f}, Y={final_pose[1]:.1f}, Z={final_pose[2]:.1f}")
    
except Exception as e:
    print(f"✗ 测试失败: {e}")
    print("可能原因: 轨迹超出工作空间范围")

### 5.2 圆弧运动测试

测试执行圆弧轨迹的能力。

In [ ]:
# 圆弧运动测试
print("=" * 70)
print("测试：圆弧运动")
print("=" * 70)
print("\n机器人将执行一个 90° 的圆弧运动")
print("  半径: 40 mm")
print("  圆弧角度: 90°")
print("  时长: 4.0 秒")

# 定义起点和终点
start_pos = [200, -40, 180, 180, 0, 0]
end_pos = [240, 0, 180, 180, 0, 0]

# 移动到起点
print("\n→ 移动到圆弧起点...")
move_robot_cartesian(start_pos, duration=2.0, wait_for_ack=True)
time.sleep(1)
print("✓ 到达起点")

# 执行圆弧
print("\n→ 执行圆弧运动...")
try:
    smooth_arc_parametric(
        end_pose=end_pos,
        radius=40,
        arc_angle=90,
        duration=4.0,
        wait_for_ack=True
    )
    time.sleep(1)
    print("✓ 圆弧运动完成")
    
    # 验证终点位置
    final_pose = get_robot_pose()
    print(f"\n目标终点: X={end_pos[0]}, Y={end_pos[1]}, Z={end_pos[2]}")
    print(f"实际终点: X={final_pose[0]:.1f}, Y={final_pose[1]:.1f}, Z={final_pose[2]:.1f}")
    
except Exception as e:
    print(f"✗ 测试失败: {e}")

### 5.3 样条曲线测试

测试通过多个路点的平滑样条曲线运动。

In [ ]:
# 样条曲线测试
print("=" * 70)
print("测试：样条曲线")
print("=" * 70)
print("\n机器人将通过多个路点执行平滑曲线")

# 定义路点
waypoints = [
    [200, 0, 150, 180, 0, 0],
    [220, 30, 170, 180, 0, 0],
    [240, 0, 190, 180, 0, 0],
    [220, -30, 170, 180, 0, 0],
    [200, 0, 150, 180, 0, 0]
]

print(f"\n路点数量: {len(waypoints)}")
print("总时长: 10.0 秒")
print("\n路点列表:")
for i, wp in enumerate(waypoints, 1):
    print(f"  {i}. X={wp[0]}, Y={wp[1]}, Z={wp[2]}")

# 执行样条曲线
print("\n→ 开始执行样条曲线...")
try:
    smooth_spline(waypoints, duration=10.0, wait_for_ack=True)
    time.sleep(1)
    print("✓ 样条曲线完成")
    
    # 验证是否返回起点
    final_pose = get_robot_pose()
    start_point = waypoints[0]
    error = ((final_pose[0]-start_point[0])**2 + 
             (final_pose[1]-start_point[1])**2 + 
             (final_pose[2]-start_point[2])**2)**0.5
    print(f"\n返回起点误差: {error:.2f} mm")
    
except Exception as e:
    print(f"✗ 测试失败: {e}")

### 5.4 螺旋运动测试（可选）

这个测试较为复杂，执行时间较长。

In [ ]:
# 螺旋运动测试
print("=" * 70)
print("测试：螺旋运动")
print("=" * 70)
print("\n机器人将执行螺旋上升运动")
print("  中心: [220, 0, 120] mm")
print("  半径: 25 mm")
print("  螺距: 15 mm（每圈上升高度）")
print("  总高度: 60 mm（约 4 圈）")
print("  时长: 15.0 秒")
print("\n这是一个较长的运动，请耐心等待...")

# 询问是否执行
RUN_HELIX = True  # 设置为 False 可跳过此测试

if RUN_HELIX:
    print("\n→ 开始执行螺旋运动...")
    try:
        smooth_helix(
            center=[220, 0, 120],
            radius=25,
            pitch=15,
            height=60,
            duration=15.0,
            wait_for_ack=True
        )
        time.sleep(1)
        print("✓ 螺旋运动完成")
        
        final_pose = get_robot_pose()
        print(f"最终高度: Z={final_pose[2]:.1f} mm")
        
    except Exception as e:
        print(f"✗ 测试失败: {e}")
else:
    print("\n跳过螺旋运动测试（RUN_HELIX = False）")

# 返回零位
print("\n→ 返回零位...")
home_robot(wait_for_ack=True)
print("✓ 已返回零位")

print("\n" + "=" * 70)
print("✓ 高级运动测试全部完成！")
print("=" * 70)

---

## 📹 阶段 6：视觉系统测试

### 6.1 检测 RealSense 相机

In [ ]:
# 检测 RealSense 相机
print("=" * 70)
print("视觉系统测试 - 相机检测")
print("=" * 70)

try:
    import pyrealsense2 as rs
    import cv2
    import numpy as np
    
    print("\n[1/3] 检测 RealSense 相机...")
    ctx = rs.context()
    devices = ctx.query_devices()
    
    if len(devices) == 0:
        print("✗ 未检测到 RealSense 相机")
        print("\n请检查:")
        print("  1. 相机是否通过 USB 3.0 连接")
        print("  2. 相机是否有电（LED 灯是否亮起）")
        print("  3. 尝试运行 realsense-viewer")
    else:
        print(f"✓ 检测到 {len(devices)} 个 RealSense 设备")
        for i, dev in enumerate(devices):
            print(f"\n设备 {i+1}:")
            print(f"  名称: {dev.get_info(rs.camera_info.name)}")
            print(f"  序列号: {dev.get_info(rs.camera_info.serial_number)}")
            print(f"  固件版本: {dev.get_info(rs.camera_info.firmware_version)}")
        
        CAMERA_AVAILABLE = True
        
except ImportError:
    print("✗ pyrealsense2 模块未安装")
    print("  安装命令: pip3 install pyrealsense2")
    CAMERA_AVAILABLE = False
except Exception as e:
    print(f"✗ 相机检测失败: {e}")
    CAMERA_AVAILABLE = False

### 6.2 初始化视觉控制器并获取图像

In [ ]:
# 初始化视觉控制器
if CAMERA_AVAILABLE:
    print("\n[2/3] 初始化视觉控制器...")
    try:
        # 导入视觉控制器
        import sys
        vision_path = os.path.join(current_dir, 'Headless', 'Vision')
        if vision_path not in sys.path:
            sys.path.insert(0, vision_path)
        
        from vision_controller import VisionController
        
        print("→ 启动相机...")
        vision = VisionController()
        vision.start()
        
        time.sleep(2)  # 等待相机稳定
        print("✓ 视觉控制器初始化成功")
        
        VISION_READY = True
        
    except Exception as e:
        print(f"✗ 初始化失败: {e}")
        VISION_READY = False
else:
    print("\n跳过视觉控制器初始化（相机不可用）")
    VISION_READY = False

In [ ]:
# 获取并显示相机图像
if VISION_READY:
    print("\n[3/3] 获取相机图像...")
    try:
        color_frame, depth_frame = vision.get_frames()
        
        if color_frame is not None and depth_frame is not None:
            print("✓ 成功获取图像")
            print(f"  彩色图像尺寸: {color_frame.shape}")
            print(f"  深度图像尺寸: {depth_frame.shape}")
            
            # 在 Jupyter 中显示图像
            from IPython.display import Image as IPImage
            from IPython.display import display
            import matplotlib.pyplot as plt
            
            # 创建深度彩色图
            depth_colormap = cv2.applyColorMap(
                cv2.convertScaleAbs(depth_frame, alpha=0.03),
                cv2.COLORMAP_JET
            )
            
            # 显示图像
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
            
            # 彩色图像
            ax1.imshow(cv2.cvtColor(color_frame, cv2.COLOR_BGR2RGB))
            ax1.set_title('彩色图像', fontsize=14)
            ax1.axis('off')
            
            # 深度图像
            ax2.imshow(cv2.cvtColor(depth_colormap, cv2.COLOR_BGR2RGB))
            ax2.set_title('深度图像', fontsize=14)
            ax2.axis('off')
            
            plt.tight_layout()
            plt.show()
            
            print("\n✓ 图像显示完成")
            
        else:
            print("✗ 无法获取图像")
            
    except Exception as e:
        print(f"✗ 获取图像失败: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\n跳过图像获取（视觉系统未就绪）")

### 6.3 检查手眼标定文件

In [ ]:
# 检查手眼标定
if VISION_READY:
    print("=" * 70)
    print("检查手眼标定")
    print("=" * 70)
    
    calib_file = vision.calibration_file
    print(f"\n标定文件路径: {calib_file}")
    
    if os.path.exists(calib_file):
        print("✓ 找到标定文件")
        
        # 加载标定数据
        calib_data = np.load(calib_file)
        print("\n标定文件包含以下数据:")
        for key in calib_data.files:
            data = calib_data[key]
            if hasattr(data, 'shape'):
                print(f"  - {key}: shape = {data.shape}")
            else:
                print(f"  - {key}: {data}")
        
        print("\n✓ 视觉引导功能可用")
        CALIBRATION_READY = True
        
    else:
        print("✗ 未找到标定文件")
        print("\n视觉引导功能需要先进行手眼标定")
        print("标定步骤:")
        print("  1. 打印标定板: python Headless/Vision/generate_charuco_board.py")
        print("  2. 运行标定:   python Headless/Vision/calibrate_hand_eye.py")
        print("  3. 验证标定:   python Headless/Vision/validate_calibration.py")
        CALIBRATION_READY = False
        
    # 关闭相机
    print("\n→ 关闭相机...")
    vision.stop()
    print("✓ 相机已关闭")
    
else:
    print("\n跳过标定检查（视觉系统未就绪）")
    CALIBRATION_READY = False

---

## 🎯 阶段 7：综合应用测试

### 7.1 简单抓放任务演示

这是一个完整的抓放流程演示。

In [ ]:
# 简单抓放任务
print("=" * 70)
print("综合应用：简单抓放任务")
print("=" * 70)
print("\n这个演示将展示一个完整的抓放流程:")
print("  1. 移动到观察位置")
print("  2. 打开夹爪")
print("  3. 移动到抓取位置")
print("  4. 关闭夹爪")
print("  5. 抬起物体")
print("  6. 移动到放置位置")
print("  7. 下降")
print("  8. 松开夹爪")
print("  9. 返回安全位置")
print("\n⚠️  确保机器人前方有足够空间！")

RUN_PICK_PLACE = True  # 设置为 False 可跳过此演示

if RUN_PICK_PLACE and GRIPPER_TYPE == 'electric':
    try:
        print("\n" + "=" * 70)
        print("开始执行抓放任务")
        print("=" * 70)
        
        # 步骤 1: 移动到观察位置
        print("\n[1/9] 移动到观察位置...")
        move_robot_joints([0, -90, 180, 0, -45, 180], duration=3.0, wait_for_ack=True)
        time.sleep(1)
        print("✓ 到达观察位置")
        
        # 步骤 2: 打开夹爪
        print("\n[2/9] 打开夹爪...")
        control_electric_gripper(action='move', position=0, speed=100, wait_for_ack=True)
        time.sleep(1)
        print("✓ 夹爪已打开")
        
        # 步骤 3: 移动到抓取位置（示例位置）
        print("\n[3/9] 移动到抓取位置...")
        pick_position = [200, 0, 100, 180, 0, 0]
        move_robot_cartesian(pick_position, duration=2.0, wait_for_ack=True)
        time.sleep(1)
        print(f"✓ 到达抓取位置: {pick_position[:3]}")
        
        # 步骤 4: 关闭夹爪抓取
        print("\n[4/9] 关闭夹爪抓取...")
        control_electric_gripper(action='move', position=200, speed=100, current=500, wait_for_ack=True)
        time.sleep(1.5)
        
        # 检查是否抓到物体
        status = get_electric_gripper_status(verbose=False)
        if status and status[5]:
            print("✓ 检测到物体被抓取")
        else:
            print("⚠ 未检测到物体（这是正常的，因为没有实际物体）")
        
        # 步骤 5: 抬起物体
        print("\n[5/9] 抬起物体...")
        lift_position = [200, 0, 200, 180, 0, 0]
        move_robot_cartesian(lift_position, duration=2.0, wait_for_ack=True)
        time.sleep(1)
        print("✓ 物体已抬起")
        
        # 步骤 6: 移动到放置位置
        print("\n[6/9] 移动到放置位置...")
        place_position = [300, 100, 200, 180, 0, 0]
        move_robot_cartesian(place_position, duration=3.0, wait_for_ack=True)
        time.sleep(1)
        print(f"✓ 到达放置位置: {place_position[:3]}")
        
        # 步骤 7: 下降
        print("\n[7/9] 下降到放置高度...")
        lower_position = [300, 100, 110, 180, 0, 0]
        move_robot_cartesian(lower_position, duration=2.0, wait_for_ack=True)
        time.sleep(1)
        print("✓ 下降完成")
        
        # 步骤 8: 松开夹爪
        print("\n[8/9] 松开夹爪...")
        control_electric_gripper(action='move', position=0, speed=100, wait_for_ack=True)
        time.sleep(1)
        print("✓ 夹爪已松开")
        
        # 步骤 9: 返回安全位置
        print("\n[9/9] 返回安全位置...")
        home_robot(wait_for_ack=True)
        time.sleep(1)
        print("✓ 返回完成")
        
        print("\n" + "=" * 70)
        print("✓ 抓放任务演示完成！")
        print("=" * 70)
        
    except Exception as e:
        print(f"\n✗ 任务执行失败: {e}")
        print("正在返回安全位置...")
        home_robot(wait_for_ack=True)
        
else:
    if not RUN_PICK_PLACE:
        print("\n跳过抓放演示（RUN_PICK_PLACE = False）")
    else:
        print("\n跳过抓放演示（需要电动夹爪）")

---

## 📊 测试总结

### 生成测试报告

In [ ]:
# 测试总结
print("=" * 70)
print("PAROL6 机械臂功能测试总结")
print("=" * 70)

print("\n✓ 已完成的测试:")
print("  ✓ 环境依赖检查")
print("  ✓ 基础连接测试")
print("  ✓ 机器人状态查询")
print("  ✓ 单关节运动")
print("  ✓ 多关节协调运动")
print("  ✓ 笛卡尔空间运动")
print("  ✓ 笛卡尔直线运动")
print("  ✓ 点动控制")
print(f"  ✓ {GRIPPER_TYPE.capitalize()} 夹爪控制")
print("  ✓ 圆形轨迹")
print("  ✓ 圆弧运动")
print("  ✓ 样条曲线")
if CAMERA_AVAILABLE:
    print("  ✓ 视觉系统测试")
if RUN_PICK_PLACE:
    print("  ✓ 综合抓放演示")

print("\n📈 系统状态:")
print(f"  核心功能:     ✓ 正常")
print(f"  运动控制:     ✓ 正常")
print(f"  夹爪控制:     ✓ 正常")
print(f"  高级运动:     ✓ 正常")
print(f"  视觉系统:     {'✓ 正常' if CAMERA_AVAILABLE else '⚠ 未检测到相机'}")
print(f"  手眼标定:     {'✓ 已完成' if CALIBRATION_READY else '⚠ 需要标定'}")

print("\n🎯 下一步建议:")
if not CAMERA_AVAILABLE:
    print("  - 连接 RealSense 相机以启用视觉功能")
if CAMERA_AVAILABLE and not CALIBRATION_READY:
    print("  - 进行手眼标定以启用视觉引导功能")
print("  - 开发自己的应用程序")
print("  - 尝试 Gemini AI 控制（运行 gemini.py）")
print("  - 优化运动参数以适应你的需求")

print("\n" + "=" * 70)
print("测试完成！系统已准备就绪。")
print("=" * 70)

# 最后确保机器人在安全位置
print("\n→ 确保机器人返回零位...")
home_robot(wait_for_ack=True)
print("✓ 机器人已在零位")

print("\n感谢使用 PAROL6 机械臂系统！🤖")

---

## 📚 参考资源

### 文档
- **中文使用说明**: `中文使用说明.md`
- **分步测试教程**: `分步测试教程.md`
- **API 详细文档**: `API-Specific-README.md`
- **项目概述**: `README.md`

### 相关链接
- **PAROL6 官方仓库**: https://github.com/PCrnjak/PAROL-commander-software
- **Discord 社区**: https://discord.com/invite/prjUvjmGpZ
- **Gemini API 文档**: https://ai.google.dev/gemini-api/docs
- **RealSense SDK**: https://github.com/IntelRealSense/librealsense

### 常用命令

```python
# 导入 API
from robot_api import *

# 基础控制
home_robot()                                    # 回零
move_robot_joints([0,0,0,0,0,0], duration=5)  # 关节运动
move_robot_pose([x,y,z,rx,ry,rz], speed=50)   # 位置运动

# 查询状态
get_robot_pose()         # 获取位置
get_robot_joint_angles() # 获取角度
get_robot_status()       # 获取完整状态

# 夹爪控制
control_electric_gripper(action='open')                    # 打开
control_electric_gripper(action='move', position=200)      # 移动
```

---

**祝你使用愉快！** 🎉